### Setup your environnement variables

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
os.environ["UIFORM_API_KEY"] = "YOUR API KEY" # go to https://uiform.com to create your API Key

In [1]:
from dotenv import load_dotenv
assert load_dotenv("../.env.local") # Load environment variables from a .env file

In [3]:
import os
os.getenv("UIFORM_API_KEY")

'sk_3ovI3g3Hq7pmBvySp3tP2OCMlZtHSNsAXfKG6VQjiDI'

In [25]:
from uiform import UiForm, Schema
from pydantic import BaseModel, Field, ConfigDict
import os

uiclient = UiForm()
doc_msg = uiclient.documents.create_messages(
    document = "freight/booking_confirmation.jpg"
)

class CalendarEvent(BaseModel):
    model_config = ConfigDict(json_schema_extra = {"X-SystemPrompt": "You are a useful assistant."})

    name: str = Field(...,
        description="The name of the calendar event.",
        json_schema_extra={"X-FieldPrompt": "Provide a descriptive and concise name for the event."}
    )
    date: str = Field(...,
        description="The date of the calendar event in ISO 8601 format.",
        json_schema_extra={
            'X-ReasoningPrompt': 'The user can mention it in any format, like **next week** or **tomorrow**. Infer the right date format from the user input.',
        }
    )

schema_obj =Schema(
    pydantic_model = CalendarEvent
)


## OpenAI

In [9]:
from openai import OpenAI

client = OpenAI()

#----

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=doc_msg.openai_messages + [
        {
            "role": "user",
            "content": "Summarize the document"
        }
    ]
)

# --- OR 

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": schema_obj.schema_version,
            "schema": schema_obj.inference_json_schema,
            "strict": True
        }
    }
)

# --- OR 

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format=schema_obj.inference_pydantic_model
)



# xAI

In [11]:
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("XAI_API_KEY"),
    base_url="https://api.x.ai/v1"
)

#----

completion = client.chat.completions.create(
    model="grok-2-vision-1212",
    messages=doc_msg.openai_messages + [
        {
            "role": "user",
            "content": "Summarize the document"
        }
    ]
)

# --- OR 

completion = client.chat.completions.create(
    model="grok-2-vision-1212",
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": schema_obj.schema_version,
            "schema": schema_obj.inference_json_schema,
            "strict": True
        }
    }
)

# --- OR 

completion = client.beta.chat.completions.parse(
    model="grok-2-vision-1212",
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format=schema_obj.inference_pydantic_model
)




# Gemini

In [12]:
from openai import OpenAI

client = OpenAI(
    api_key = os.getenv("GEMINI_API_KEY"),
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
)

#----

completion = client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=doc_msg.openai_messages + [
        {
            "role": "user",
            "content": "Summarize the document"
        }
    ]
)


# --- OR 

completion = client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=schema_obj.openai_messages + doc_msg.openai_messages + [
        {
            "role": "user",
            "content": "Extract informations from this document."
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": schema_obj.schema_version,
            "schema": schema_obj.inference_gemini_json_schema,
            "strict": True
        }
    }
)

# --- Does not work with beta.chat.completions.parse !

# Anthropic

In [9]:
from anthropic import Anthropic

client = Anthropic(api_key=os.getenv("CLAUDE_API_KEY"))

In [27]:
from anthropic import Anthropic

message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1000,
    temperature=0,
    system = doc_msg.anthropic_system_prompt
    messages=doc_msg.anthropic_messages + [
        {
            "role": "user",
            "content": "Summarize this Document"
        }
    ],
)
print(message.model_dump())

{'id': 'msg_0114HR8sPeHAEedhVLrfakzT', 'content': [{'text': "This is a French freight confirmation document (Confirmation d'affrétement) between ACME Corporation and TRANSPORT EXPRESS for a shipping order. Here are the key details:\n\n1. Shipping Reference Numbers:\n- Order #: SHIP-001\n- Client Ref: BC-67890\n\n2. Parties Involved:\n- Shipper: ACME Corporation (London, UK)\n- Carrier: TRANSPORT EXPRESS (Paris, France)\n- Delivery to: Beta Industries (Munich, Germany)\n\n3. Cargo Details:\n- Weight: 500T\n- 5 Pallets, 10 Units\n- Volume: 1.44 M3\n- Temperature requirement: 5°C - 25°C (Cool)\n- Dangerous goods: UN1202 Class 3 - Flammable liquid\n\n4. Timeline:\n- Pickup: 5/2/2023 (8:00-12:00)\n- Delivery: 5/3/2023 (10:00-16:00)\n\n5. Price: 1500 EUR (excluding VAT), including fuel costs\n\n6. Special Conditions:\n- Carrier must have specific insurance for sensitive goods\n- Must follow security protocols\n- Must handle customs documentation properly\n- Must report any incidents immediat

In [18]:
len(doc_msg.anthropic_messages[0]['content'][0])

2